In [ ]:
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    import os
    import pickle
    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )

    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)

    estimator.fit(x_train, y_train)
    y_pred = estimator.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if verbose > 0:
        print(estimator, ":", sep="")
        print(f"  MSE: {mse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

    if not os.path.exists("estimator.pickle"):
        saved_estimator = None
    else:
        with open("estimator.pickle", "rb") as file:
            saved_estimator = pickle.load(file)

    if saved_estimator is None or estimator.score(
        x_test, y_test
    ) > saved_estimator.score(x_test, y_test):
        with open("estimator.pickle", "wb") as file:
            pickle.dump(estimator, file)


In [16]:
train_estimator(0.2, 0.2)

ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.4386911951894716
  MAE: 0.5236106762028768
  R2: 0.2822387414965033


In [17]:
def use_estimator():

    import pandas as pd
    import pickle

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    y_pred = estimator.predict(x)

    return y_pred

use_estimator()

array([5.24090703, 5.31521455, 5.33699336, ..., 5.80575678, 5.50547416,
       5.78406716], shape=(1599,))

In [18]:
def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y

In [19]:
def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )
    return x_train, x_test, y_train, y_test    

In [20]:
def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2
    

In [21]:
def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    

In [22]:
def save_best_estimator(estimator):

    import os
    import pickle

    with open("estimator.pickle", "wb") as file:
        pickle.dump(estimator, file)

In [23]:
def load_best_estimator():

    import os
    import pickle

    if not os.path.exists("estimator.pickle"):
        return None
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    return estimator

In [24]:
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    from sklearn.linear_model import ElasticNet

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    estimator.fit(x_train, y_train)
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)
    

In [25]:
def check_estimator():

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = load_best_estimator()
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    report(estimator, mse, mae, r2)

In [26]:
def make_hyperparameters_search(alphas, l1_ratios):

    for alpha in alphas:
        for l1_ratio in l1_ratios:
            train_estimator(alpha=alpha, l1_ratio=l1_ratio, verbose=0)

In [27]:
def train_estimator(alphas, l1_ratios, n_splits=5, verbose=1):

    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    # -------------------------------------------------------------------------
    # Búsqueda de parámetros con validación cruzada
    #
    estimator = GridSearchCV(
        estimator=ElasticNet(
            random_state=12345,
        ),
        param_grid={
            "alpha": alphas,
            "l1_ratio": l1_ratios,
        },
        cv=n_splits,
        refit=True,
        verbose=0,
        return_train_score=False,
    )
    # -------------------------------------------------------------------------

    estimator.fit(x_train, y_train)

    estimator = estimator.best_estimator_

    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)

In [29]:
import numpy as np

train_estimator(np.linspace(0,0.5,10), np.linspace(0,0.5,10))

d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\base.py:1363: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.006e+02, tolerance: 6.582e-02
Linear regression m

ElasticNet(alpha=np.float64(0.0), l1_ratio=np.float64(0.0), random_state=12345):
  MSE: 0.4000725270850549
  MAE: 0.48447480789400293
  R2: 0.34542438124562813


d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\base.py:1363: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
d:\UNAL\2025-1S\Fundamentos de analitica\PRE-20-ajuste-de-hiperparametros-JhofredCam\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.539e+02, tolerance: 7.975e-02
Linear regression m

In [30]:
check_estimator()

ElasticNet(alpha=np.float64(0.0), l1_ratio=np.float64(0.0), random_state=12345):
  MSE: 0.4000725270850549
  MAE: 0.48447480789400293
  R2: 0.34542438124562813
